# Replace Google Metadata

In [30]:
import subprocess
import json
from pathlib import Path

def get_metadata(video_path: Path) -> dict:
    # Ensure the video file exists
    if not video_path.exists():
        raise FileNotFoundError(f"Video file not found: {video_path}")

    try:
        result = subprocess.run(
            [
                'ffprobe', '-v', 'quiet', '-print_format', 'json',
                '-show_format',     # Format-level metadata
                '-show_streams',    # Stream-specific metadata
                '-show_chapters',   # Chapter metadata
                '-show_programs',   # Program metadata
                str(video_path)
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            timeout=10  # Set a timeout of 10 seconds
        )

        # Check if ffprobe command was successful
        if result.returncode != 0:
            raise RuntimeError(f"ffprobe failed with error: {result.stderr}")

        # Parse JSON output
        if not result.stdout:
            raise ValueError("ffprobe produced no output")
        
        return json.loads(result.stdout)

    except subprocess.TimeoutExpired:
        raise TimeoutError(f"ffprobe timed out while processing {video_path}")
    except json.JSONDecodeError:
        raise ValueError(f"Failed to parse ffprobe output as JSON for {video_path}")
    except FileNotFoundError:
        raise FileNotFoundError("ffprobe is not installed or not found in PATH")

def clean_google_metadata(input_path: Path, output_path: Path) -> None:
    # Ensure the input video file exists
    if not input_path.exists():
        raise FileNotFoundError(f"Input video file not found: {input_path}")

    # Ensure the output directory exists
    output_path.parent.mkdir(parents=True, exist_ok=True)

    try:
        # Run ffmpeg to strip all metadata and copy video/audio streams
        result = subprocess.run(
            [
                'ffmpeg', '-i', str(input_path),  # Input file
                '-map_metadata', '-1',            # Remove all metadata
                '-c:v', 'copy',                   # Copy video stream
                '-c:a', 'copy',                   # Copy audio stream
                '-y',                             # Overwrite output file if it exists
                str(output_path)                  # Output file
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            timeout=30  # Set a timeout of 30 seconds
        )

        # Check if ffmpeg command was successful
        if result.returncode != 0:
            raise RuntimeError(f"ffmpeg failed with error: {result.stderr}")

    except subprocess.TimeoutExpired:
        raise TimeoutError(f"ffmpeg timed out while processing {input_path}")
    except FileNotFoundError:
        raise FileNotFoundError("ffmpeg is not installed or not found in PATH")

# Example usage
try:
    input_video = Path("mp4_files/asmr.mp4")
    output_video = Path("mp4_files_clean/asmr_clean.mp4")
    
    # Get original metadata
    metadata = get_metadata(input_video)
    print("Original metadata:", metadata)
    
    # Clean metadata and save to new file
    clean_google_metadata(input_video, output_video)
    print(f"Cleaned video saved to {output_video}")
    
    # Get metadata of cleaned video to verify
    cleaned_metadata = get_metadata(output_video)
    print("Cleaned metadata:", cleaned_metadata)
    
except Exception as e:
    print(f"Error: {e}")

Original metadata: {'programs': [], 'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'Constrained Baseline', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 1280, 'height': 720, 'coded_width': 1280, 'coded_height': 720, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 0, 'sample_aspect_ratio': '1:1', 'display_aspect_ratio': '16:9', 'pix_fmt': 'yuv420p', 'level': 31, 'color_range': 'tv', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '24/1', 'avg_frame_rate': '24/1', 'time_base': '1/12288', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 98304, 'duration': '8.000000', 'bit_rate': '876743', 'bits_per_raw_sample': '8', 'nb_frames': '192', 'extradata_size': 43, 'disposition': {'default': 1, 'dub':